In [1]:
from time import perf_counter

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [2]:
model_path = "outputs"

device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path).to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 1920)
    (wpe): Embedding(1024, 1920)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
     

In [85]:
input_text = "추억을 회상한다."
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

with torch.no_grad():
    # Check generation time
    t = perf_counter()

    output = model.generate(
        input_ids,
        max_length=64,
        repetition_penalty=2.0,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
        do_sample=True,
        top_k=30,
        top_p=0.95,
    )
    generated_text = tokenizer.decode(output[0])

    spent = perf_counter() - t

    print("generated text:", generated_text, sep="\n")
    print(f"\ntime spent: {spent:.2f} sec")

generated text:
추억을 회상한다.그때의 기억들은 지금도 그립지만, 그때보다는 더 나아진 것 같기도 하다. 지금에야 <unk>미안해 그리고 고마워, 사랑한다 내 사람들아!......라고 말하지만, 그때는 정말 그게 전부인 줄 알았으니까. 물론 가끔은 다른 사람을 만나보고 싶다고 생각하면서도, 그럴 때

time spent: 1.28 sec
